In [5]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

In [12]:
X_df = pd.read_csv("drive/MyDrive/PPI/processed_protein_pairs_df.csv")
y_df = pd.read_csv("drive/MyDrive/PPI/not_shuffled_targets.csv")
X_df = X_df.astype('float32')
# X_train, X_test = train_test_split(X_df,test_size=0.2)
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')

print(X_df.shape)

(104738, 1024)


In [4]:
X_df.iloc[0].shape

(1024,)

In [ ]:
encoder_input = keras.Input(shape=(1024,), name='protein_pair')
x = keras.layers.Flatten()(encoder_input)
encoder_output = keras.layers.Dense(256, activation="relu")(x)
encoder = keras.Model(encoder_input, encoder_output, name='encoder')
decoder_input = keras.layers.Dense(256, activation="relu")(encoder_output)
x = keras.layers.Dense(1024, activation="relu")(decoder_input)
decoder_output = keras.layers.Reshape((1024,))(x)
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
autoencoder = keras.Model(encoder_input, decoder_output, name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
protein_pair (InputLayer)    [(None, 1024)]            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              263168    
_________________________________________________________________
reshape_3 (Reshape)          (None, 1024)              0         
Total params: 591,360
Trainable params: 591,360
Non-trainable params: 0
_________________________________________________

In [ ]:
autoencoder.compile(opt, loss="mse")
autoencoder.fit(X_df, X_df, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
2357/2357 [==============================] - 6s 2ms/step - loss: 0.0083 - val_loss: 0.0047
Epoch 2/5
2357/2357 [==============================] - 5s 2ms/step - loss: 0.0046 - val_loss: 0.0043
Epoch 3/5
2357/2357 [==============================] - 5s 2ms/step - loss: 0.0043 - val_loss: 0.0042
Epoch 4/5
2357/2357 [==============================] - 5s 2ms/step - loss: 0.0042 - val_loss: 0.0041
Epoch 5/5
2357/2357 [==============================] - 5s 2ms/step - loss: 0.0040 - val_loss: 0.0041


In [ ]:
compressed_df = pd.DataFrame(encoder.predict([X_df]))
compressed_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,0.0,0.632995,0.0,0.690174,0.0,0.470387,0.0,0.303314,0.342761,0.583491,0.237609,0.0,0.573898,0.0,0.0,0.581814,0.230062,0.372623,0.0,0.743237,0.200365,0.701954,0.229806,0.544221,0.359519,0.0,0.612796,0.488913,0.632268,0.0,0.705564,0.0,0.000000,0.0,0.397231,0.757876,0.292288,0.830966,0.0,0.0,...,0.452938,0.790367,0.777822,0.173677,0.0,0.0,0.570318,0.0,0.0,0.325832,0.411696,0.359135,0.442973,0.604913,0.497125,0.0,0.068725,0.768432,0.934472,0.0,0.504662,0.0,0.224459,0.382426,0.498707,0.0,0.271558,0.0,0.0,0.0,0.0,0.484029,0.0,0.0,0.0,0.370346,0.0,0.630385,0.647285,0.0
1,0.0,0.622655,0.0,0.649570,0.0,0.679950,0.0,0.150420,0.537173,0.746172,0.700076,0.0,0.626242,0.0,0.0,0.737619,0.526731,0.500097,0.0,0.656251,0.228348,0.482404,0.289642,0.569816,0.495423,0.0,0.466842,0.437372,0.481427,0.0,0.566695,0.0,0.089475,0.0,0.609309,0.632814,0.394950,1.076714,0.0,0.0,...,0.275038,0.854969,0.967526,0.215567,0.0,0.0,0.845619,0.0,0.0,0.319034,0.492466,0.389575,0.588936,0.720487,0.757494,0.0,0.088334,0.730386,1.082657,0.0,0.381281,0.0,0.399413,0.417554,0.414514,0.0,0.280411,0.0,0.0,0.0,0.0,0.313787,0.0,0.0,0.0,0.327488,0.0,0.757835,0.981023,0.0
2,0.0,0.785742,0.0,0.520621,0.0,0.496548,0.0,0.170973,0.402512,0.550968,0.213303,0.0,0.511901,0.0,0.0,0.672304,0.376046,0.321840,0.0,0.471660,0.227400,0.565728,0.091271,0.344651,0.308028,0.0,0.417532,0.481787,0.613784,0.0,0.513914,0.0,0.159745,0.0,0.496374,0.726049,0.312419,0.809623,0.0,0.0,...,0.484317,0.623242,0.854449,0.367703,0.0,0.0,0.364700,0.0,0.0,0.330018,0.329314,0.263046,0.114807,0.359697,0.593635,0.0,0.105253,0.550064,0.913285,0.0,0.309986,0.0,0.344495,0.620088,0.477012,0.0,0.212547,0.0,0.0,0.0,0.0,0.096812,0.0,0.0,0.0,0.128145,0.0,0.451081,0.330009,0.0
3,0.0,0.675157,0.0,0.574182,0.0,0.226873,0.0,0.091899,0.367461,0.485967,0.405789,0.0,0.467364,0.0,0.0,0.618798,0.235217,0.390753,0.0,0.560775,0.209970,0.551912,0.162804,0.432866,0.396987,0.0,0.406423,0.487004,0.507619,0.0,0.482190,0.0,0.161493,0.0,0.401883,0.626927,0.179655,0.725155,0.0,0.0,...,0.319942,0.586395,0.840987,0.187229,0.0,0.0,0.476047,0.0,0.0,0.139112,0.386347,0.428302,0.111098,0.550833,0.527576,0.0,0.053173,0.682731,1.083998,0.0,0.463142,0.0,0.426661,0.289260,0.379944,0.0,0.177130,0.0,0.0,0.0,0.0,0.193434,0.0,0.0,0.0,0.249931,0.0,0.426981,0.497049,0.0
4,0.0,0.755466,0.0,0.604557,0.0,0.473316,0.0,0.203912,0.436935,0.738216,0.342648,0.0,0.540919,0.0,0.0,0.785037,0.356966,0.450425,0.0,0.505476,0.264318,0.581975,0.130905,0.348917,0.388292,0.0,0.439914,0.297459,0.581194,0.0,0.513879,0.0,0.124733,0.0,0.389789,0.558916,0.241196,0.710401,0.0,0.0,...,0.423266,0.779563,0.904663,0.354884,0.0,0.0,0.520362,0.0,0.0,0.145821,0.380843,0.372768,0.278223,0.502713,0.535986,0.0,0.050503,0.617281,0.983273,0.0,0.439118,0.0,0.413044,0.333290,0.444218,0.0,0.120831,0.0,0.0,0.0,0.0,0.266127,0.0,0.0,0.0,0.261521,0.0,0.428327,0.614886,0.0


In [ ]:
# compressed_df.to_csv('drive/MyDrive/PPI/compressed_df.csv', header=True, index=False)
compressed_df.shape

(104738, 256)

In [ ]:
folds = StratifiedKFold(n_splits=2, shuffle=True)
svc_roc_auc_scores = []
svc_average_precision_score = []
svc_f1_score = []
counter = 0

for train_index, test_index in folds.split(np.array(compressed_df), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(compressed_df)[train_index], np.array(compressed_df)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = SVC()
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    svc_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    svc_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    svc_f1_score.append(f1_score(y_test, test_predictions_np))

print("svc_roc_auc_scores:", svc_roc_auc_scores)
print("Average roc_auc_score:", sum(svc_roc_auc_scores) / len(svc_roc_auc_scores))
print("svc_average_precision_score:", svc_average_precision_score)
print("Average average_precision_score:", sum(svc_average_precision_score) / len(svc_average_precision_score))
print("svc_f1_score:", svc_f1_score)
print("Average f1_score:", sum(svc_f1_score) / len(svc_f1_score))

model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
svc_roc_auc_scores: [0.7375754381340646, 0.7323588742082369]
Average roc_auc_score: 0.7349671561711507
svc_average_precision_score: [0.6873127813866047, 0.6833439403684186]
Average average_precision_score: 0.6853283608775116
svc_f1_score: [0.7122186158517433, 0.7032394664408215]
Average f1_score: 0.7077290411462824


In [7]:
encoder_input_2 = keras.Input(shape=(1024,), name='protein_pair')
x_2 = keras.layers.Flatten()(encoder_input_2)
encoder_output_2 = keras.layers.Dense(128, activation="relu")(x_2)
encoder_2 = keras.Model(encoder_input_2, encoder_output_2, name='encoder')
decoder_input_2 = keras.layers.Dense(128, activation="relu")(encoder_output_2)
x_2 = keras.layers.Dense(1024, activation="relu")(decoder_input_2)
decoder_output_2 = keras.layers.Reshape((1024,))(x_2)
opt_2 = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
autoencoder_2 = keras.Model(encoder_input_2, decoder_output_2, name='autoencoder')
autoencoder_2.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
protein_pair (InputLayer)    [(None, 1024)]            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              132096    
_________________________________________________________________
reshape_1 (Reshape)          (None, 1024)              0         
Total params: 279,808
Trainable params: 279,808
Non-trainable params: 0
_________________________________________________

In [10]:
autoencoder_2.compile(opt_2, loss="mse")
autoencoder_2.fit(X_df, X_df, epochs=5, batch_size=32, validation_split=0.1)
compressed_df_2 = pd.DataFrame(encoder_2.predict([X_df]))
compressed_df_2.head()

Epoch 1/5
2946/2946 [==============================] - 6s 2ms/step - loss: 0.0057 - val_loss: 0.0062
Epoch 2/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0057 - val_loss: 0.0062
Epoch 3/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0056 - val_loss: 0.0061
Epoch 4/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0056 - val_loss: 0.0061
Epoch 5/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0056 - val_loss: 0.0061


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.398272,0.0,0.447914,0.657565,0.454405,0.0,0.0,0.0,0.0,0.0,0.344487,0.568577,0.665389,0.414415,0.0,0.649106,0.0,0.0,0.409929,0.504027,0.380431,0.456638,0.829406,0.0,0.0,0.628768,0.0,0.0,0.410005,0.392238,0.0,0.431103,0.243705,0.480782,0.192576,0.753635,0.765443,0.491844,0.842495,0.608842,...,0.0,0.0,0.0,0.175194,0.0,0.310656,0.0,0.312249,0.0,0.125646,0.249348,0.0,0.0,0.949646,0.0,0.344050,0.498477,0.0,0.0,0.271070,0.000000,0.935814,0.386687,0.413590,0.305851,0.332969,0.591513,0.780915,1.030390,0.0,0.618044,0.695860,0.392201,0.813209,0.228728,0.671661,0.242162,0.676788,0.037126,0.0
1,0.544493,0.0,0.421228,0.797691,0.638704,0.0,0.0,0.0,0.0,0.0,0.438635,0.447537,0.614737,0.497710,0.0,0.595611,0.0,0.0,0.202621,0.522769,0.032545,0.814802,0.572402,0.0,0.0,0.672252,0.0,0.0,0.400297,0.399838,0.0,0.229881,0.614612,0.242896,0.210996,0.802403,0.735616,0.533538,0.864171,0.386906,...,0.0,0.0,0.0,0.216097,0.0,0.107387,0.0,0.511331,0.0,0.213070,0.278066,0.0,0.0,1.149123,0.0,0.458972,0.412240,0.0,0.0,0.000000,0.088135,1.030114,0.505632,0.933255,0.526515,0.083281,0.737605,0.773092,1.092437,0.0,0.659681,0.659064,0.435209,0.845401,0.342628,0.604385,0.319992,0.669042,0.301230,0.0
2,0.493133,0.0,0.464818,0.712764,0.341904,0.0,0.0,0.0,0.0,0.0,0.462341,0.427050,0.354194,0.298794,0.0,0.411513,0.0,0.0,0.495422,0.580798,0.248573,0.822228,0.868884,0.0,0.0,0.807689,0.0,0.0,0.508844,0.620183,0.0,0.435162,0.311380,0.319594,0.159895,0.698222,0.537852,0.342719,0.652206,0.675015,...,0.0,0.0,0.0,0.209454,0.0,0.172562,0.0,0.401653,0.0,0.209726,0.256927,0.0,0.0,0.722496,0.0,0.224446,0.300949,0.0,0.0,0.149691,0.000000,1.163658,0.229546,0.379019,0.398385,0.287720,0.563846,0.846016,0.772756,0.0,0.622746,0.503998,0.342715,0.739987,0.403064,0.723243,0.160192,0.693867,0.101121,0.0
3,0.184193,0.0,0.397460,0.482623,0.538862,0.0,0.0,0.0,0.0,0.0,0.435705,0.285018,0.314592,0.326154,0.0,0.510626,0.0,0.0,0.351609,0.446504,0.296234,0.498316,0.847153,0.0,0.0,0.586503,0.0,0.0,0.359619,0.527299,0.0,0.353906,0.309475,0.239889,0.171631,0.644356,0.434940,0.489988,0.605689,0.430005,...,0.0,0.0,0.0,0.212459,0.0,0.237624,0.0,0.530725,0.0,0.000000,0.179969,0.0,0.0,0.446040,0.0,0.267209,0.329505,0.0,0.0,0.000434,0.000000,1.201117,0.266387,0.436142,0.499956,0.220566,0.581281,0.716712,0.728771,0.0,0.612153,0.386894,0.307472,0.719636,0.171958,0.616202,0.106874,0.798983,0.209900,0.0
4,0.247487,0.0,0.399708,0.644653,0.367640,0.0,0.0,0.0,0.0,0.0,0.493913,0.383705,0.466551,0.373878,0.0,0.577822,0.0,0.0,0.399377,0.609160,0.209916,0.624907,0.966106,0.0,0.0,0.616005,0.0,0.0,0.482686,0.545274,0.0,0.429760,0.285132,0.241227,0.210475,0.601497,0.528053,0.406960,0.856238,0.507694,...,0.0,0.0,0.0,0.137464,0.0,0.153051,0.0,0.604308,0.0,0.086830,0.132108,0.0,0.0,0.883803,0.0,0.316998,0.430294,0.0,0.0,0.024237,0.055524,1.093544,0.377976,0.510898,0.442232,0.284761,0.620169,0.672565,0.966934,0.0,0.576607,0.635848,0.360278,0.778768,0.265541,0.532879,0.168554,0.639135,0.171149,0.0


In [13]:
folds = StratifiedKFold(n_splits=2, shuffle=True)
svc_roc_auc_scores = []
svc_average_precision_score = []
svc_f1_score = []

for train_index, test_index in folds.split(np.array(compressed_df_2), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(compressed_df_2)[train_index], np.array(compressed_df_2)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = SVC()
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    svc_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    svc_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    svc_f1_score.append(f1_score(y_test, test_predictions_np))

print("svc_roc_auc_scores:", svc_roc_auc_scores)
print("Average roc_auc_score:", sum(svc_roc_auc_scores) / len(svc_roc_auc_scores))
print("svc_average_precision_score:", svc_average_precision_score)
print("Average average_precision_score:", sum(svc_average_precision_score) / len(svc_average_precision_score))
print("svc_f1_score:", svc_f1_score)
print("Average f1_score:", sum(svc_f1_score) / len(svc_f1_score))

model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
svc_roc_auc_scores: [0.7267868449487568, 0.7268785717982013]
Average roc_auc_score: 0.726832708373479
svc_average_precision_score: [0.6777369230034374, 0.6771485352440556]
Average average_precision_score: 0.6774427291237465
svc_f1_score: [0.6963368564030732, 0.6978473498531804]
Average f1_score: 0.6970921031281267


In [15]:
encoder_input_3 = keras.Input(shape=(1024,), name='protein_pair')
x_3 = keras.layers.Flatten()(encoder_input_3)
encoder_output_3 = keras.layers.Dense(512, activation="relu")(x_3)
encoder_3 = keras.Model(encoder_input_3, encoder_output_3, name='encoder')
decoder_input_3 = keras.layers.Dense(512, activation="relu")(encoder_output_3)
x_3 = keras.layers.Dense(1024, activation="relu")(decoder_input_3)
decoder_output_3 = keras.layers.Reshape((1024,))(x_3)
opt_3 = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
autoencoder_3 = keras.Model(encoder_input_3, decoder_output_3, name='autoencoder')
autoencoder_3.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
protein_pair (InputLayer)    [(None, 1024)]            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              525312    
_________________________________________________________________
reshape_3 (Reshape)          (None, 1024)              0         
Total params: 1,312,768
Trainable params: 1,312,768
Non-trainable params: 0
_____________________________________________

In [16]:
autoencoder_3.compile(opt_3, loss="mse")
autoencoder_3.fit(X_df, X_df, epochs=5, batch_size=32, validation_split=0.1)
compressed_df_3 = pd.DataFrame(encoder_3.predict([X_df]))
compressed_df_3.head()

Epoch 1/5
2946/2946 [==============================] - 6s 2ms/step - loss: 0.0060 - val_loss: 0.0025
Epoch 2/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0022 - val_loss: 0.0022
Epoch 3/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0020 - val_loss: 0.0022
Epoch 4/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 5/5
2946/2946 [==============================] - 5s 2ms/step - loss: 0.0019 - val_loss: 0.0021


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0.0,0.368143,0.0,0.369577,0.414134,0.0,0.0,0.471692,0.0,0.341015,0.0,0.391461,0.487569,0.469379,0.311999,0.393685,0.336301,0.408628,0.355665,0.0,0.459234,0.052824,0.384813,0.265894,0.354071,0.0,0.0,0.415377,0.531776,0.624585,0.0,0.352835,0.428952,0.0,0.357756,0.346545,0.274966,0.439531,0.0,0.432141,...,0.546894,0.522989,0.0,0.0,0.0,0.281034,0.198022,0.0,0.386295,0.195473,0.241283,0.0,0.0,0.354518,0.371339,0.0,0.648465,0.387723,0.474978,0.614177,0.0,0.0,0.181540,0.0,0.514018,0.0,0.0,0.0,0.0,0.833798,0.487869,0.0,0.292795,0.047999,0.0,0.464577,0.260374,0.568009,0.197297,0.0
1,0.0,0.622014,0.0,0.116937,0.330220,0.0,0.0,0.491327,0.0,0.344232,0.0,0.613845,0.356168,0.544563,0.316435,0.682609,0.376986,0.280463,0.327266,0.0,0.330741,0.040774,0.410277,0.181414,0.271252,0.0,0.0,0.318769,0.535302,0.782163,0.0,0.449250,0.256141,0.0,0.337459,0.384920,0.363519,0.460849,0.0,0.544719,...,0.509848,0.659376,0.0,0.0,0.0,0.282556,0.175978,0.0,0.496000,0.472160,0.406307,0.0,0.0,0.260123,0.511725,0.0,0.382289,0.364293,0.475025,0.697122,0.0,0.0,0.241147,0.0,0.518794,0.0,0.0,0.0,0.0,0.700732,0.331373,0.0,0.246114,0.144527,0.0,0.454109,0.353247,0.594980,0.214988,0.0
2,0.0,0.365071,0.0,0.333982,0.509819,0.0,0.0,0.552823,0.0,0.207882,0.0,0.430195,0.423531,0.483150,0.415809,0.459366,0.416131,0.171425,0.284124,0.0,0.591719,0.130811,0.296684,0.073468,0.372108,0.0,0.0,0.322272,0.483719,0.526229,0.0,0.442209,0.288760,0.0,0.150725,0.497421,0.353031,0.396359,0.0,0.651187,...,0.791603,0.455199,0.0,0.0,0.0,0.309954,0.288245,0.0,0.549633,0.327824,0.326333,0.0,0.0,0.340883,0.198599,0.0,0.531424,0.480272,0.432944,0.207055,0.0,0.0,0.182729,0.0,0.546230,0.0,0.0,0.0,0.0,0.585329,0.457448,0.0,0.154797,0.000000,0.0,0.212779,0.296058,0.512357,0.252421,0.0
3,0.0,0.483689,0.0,0.233102,0.376468,0.0,0.0,0.403232,0.0,0.282397,0.0,0.447551,0.457635,0.420619,0.402320,0.469084,0.480966,0.276588,0.281684,0.0,0.294235,0.042822,0.437379,0.172306,0.194761,0.0,0.0,0.305246,0.463836,0.491859,0.0,0.384142,0.249157,0.0,0.229196,0.361376,0.232976,0.458546,0.0,0.549308,...,0.563854,0.396922,0.0,0.0,0.0,0.241360,0.232840,0.0,0.524603,0.237200,0.285194,0.0,0.0,0.382285,0.313235,0.0,0.431800,0.462970,0.599797,0.305706,0.0,0.0,0.213794,0.0,0.516906,0.0,0.0,0.0,0.0,0.591089,0.338894,0.0,0.151336,0.060212,0.0,0.249695,0.108435,0.501253,0.186209,0.0
4,0.0,0.456680,0.0,0.424594,0.385337,0.0,0.0,0.540070,0.0,0.265826,0.0,0.396904,0.499368,0.498950,0.457542,0.615498,0.311745,0.326704,0.383960,0.0,0.410370,0.000000,0.304057,0.273004,0.330563,0.0,0.0,0.371488,0.516666,0.544134,0.0,0.473942,0.206238,0.0,0.163901,0.393547,0.314505,0.464624,0.0,0.528479,...,0.469161,0.545354,0.0,0.0,0.0,0.251794,0.254498,0.0,0.475091,0.281477,0.224775,0.0,0.0,0.320496,0.373935,0.0,0.391371,0.470884,0.422193,0.215702,0.0,0.0,0.193512,0.0,0.585662,0.0,0.0,0.0,0.0,0.624061,0.394513,0.0,0.256364,0.131878,0.0,0.335776,0.170942,0.481960,0.321344,0.0


In [17]:
folds = StratifiedKFold(n_splits=2, shuffle=True)
svc_roc_auc_scores = []
svc_average_precision_score = []
svc_f1_score = []

for train_index, test_index in folds.split(np.array(compressed_df_3), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(compressed_df_3)[train_index], np.array(compressed_df_3)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = SVC()
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    svc_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    svc_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    svc_f1_score.append(f1_score(y_test, test_predictions_np))

print("svc_roc_auc_scores:", svc_roc_auc_scores)
print("Average roc_auc_score:", sum(svc_roc_auc_scores) / len(svc_roc_auc_scores))
print("svc_average_precision_score:", svc_average_precision_score)
print("Average average_precision_score:", sum(svc_average_precision_score) / len(svc_average_precision_score))
print("svc_f1_score:", svc_f1_score)
print("Average f1_score:", sum(svc_f1_score) / len(svc_f1_score))

model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
model: SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)
X_train legth: 52369
X_test legth: 52369
svc_roc_auc_scores: [0.7478675631469214, 0.7493349717727414]
Average roc_auc_score: 0.7486012674598315
svc_average_precision_score: [0.6964628802723514, 0.6981209644267227]
Average average_precision_score: 0.6972919223495371
svc_f1_score: [0.7270095931194177, 0.7284612043108619]
Average f1_score: 0.7277353987151398


In [3]:
all_df = pd.read_csv("drive/MyDrive/PPI/processed_protein_pairs_df.csv")
all_df = all_df.astype('float32')
y_df = pd.read_csv("drive/MyDrive/PPI/not_shuffled_targets.csv")

folds = StratifiedKFold(n_splits=3, shuffle=True)
roc_auc_scores = []
average_precision_score = []
f1_score = []

for train_index, test_index in folds.split(np.array(compressed_df), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(compressed_df)[train_index], np.array(compressed_df)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = KNeighborsClassifier(n_neighbors=7)
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    f1_score.append(f1_score(y_test, test_predictions_np))
    print("roc_auc_score :", roc_auc_score(y_test, test_predictions_np.ravel()))
    print("average_precision_score: ", average_precision_score(y_test, test_predictions_np))
    print("f1_score: ", f1_score(y_test, test_predictions_np))

print("roc_auc_scores:", roc_auc_scores)
print("Average roc_auc_score:", sum(roc_auc_scores) / len(roc_auc_scores))
print("average_precision_score:", average_precision_score)
print("Average average_precision_score:", sum(average_precision_score) / len(average_precision_score))
print("knn_f1_score:", f1_score)
print("Average f1_score:", sum(f1_score) / len(knn_f1_score))

NameError: ignored

In [7]:
all_df = pd.read_csv("drive/MyDrive/PPI/processed_protein_pairs_df.csv")
all_df = all_df.astype('float32')
y_df = pd.read_csv("drive/MyDrive/PPI/not_shuffled_targets.csv")

folds = StratifiedKFold(n_splits=3, shuffle=True)
rf_roc_auc_scores = []
rf_average_precision_score = []
rf_f1_score = []

for train_index, test_index in folds.split(np.array(all_df), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(all_df)[train_index], np.array(all_df)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = RandomForestClassifier()
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    rf_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    rf_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    rf_f1_score.append(f1_score(y_test, test_predictions_np))
    print("roc_auc_score :", roc_auc_score(y_test, test_predictions_np.ravel()))
    print("rf_average_precision_score: ", average_precision_score(y_test, test_predictions_np))
    print("rf_f1_score: ", f1_score(y_test, test_predictions_np))

print("rf_roc_auc_scores:", rf_roc_auc_scores)
print("Average roc_auc_score:", sum(rf_roc_auc_scores) / len(rf_roc_auc_scores))
print("rf_average_precision_score:", rf_average_precision_score)
print("Average rf_average_precision_score:", sum(rf_average_precision_score) / len(rf_average_precision_score))
print("rf_f1_score:", rf_f1_score)
print("Average rf_f1_score:", sum(rf_f1_score) / len(rf_f1_score))

model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
X_train legth: 69825
X_test legth: 34913
roc_auc_score : 0.8268850400226572
rf_average_precision_score:  0.7789444546081069
rf_f1_score:  0.820161866222328
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=N

In [8]:
folds = StratifiedKFold(n_splits=3, shuffle=True)
rf_roc_auc_scores = []
rf_average_precision_score = []
rf_f1_score = []

for train_index, test_index in folds.split(np.array(all_df), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(all_df)[train_index], np.array(all_df)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = RandomForestClassifier(n_estimators=200)
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    rf_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    rf_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    rf_f1_score.append(f1_score(y_test, test_predictions_np))
    print("roc_auc_score :", roc_auc_score(y_test, test_predictions_np.ravel()))
    print("rf_average_precision_score: ", average_precision_score(y_test, test_predictions_np))
    print("rf_f1_score: ", f1_score(y_test, test_predictions_np))

print("rf_roc_auc_scores:", rf_roc_auc_scores)
print("Average roc_auc_score:", sum(rf_roc_auc_scores) / len(rf_roc_auc_scores))
print("rf_average_precision_score:", rf_average_precision_score)
print("Average rf_average_precision_score:", sum(rf_average_precision_score) / len(rf_average_precision_score))
print("rf_f1_score:", rf_f1_score)
print("Average rf_f1_score:", sum(rf_f1_score) / len(rf_f1_score))

model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
X_train legth: 69825
X_test legth: 34913
roc_auc_score : 0.8305799576432062
rf_average_precision_score:  0.7835859182148002
rf_f1_score:  0.8238797082030669
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=

In [9]:
folds = StratifiedKFold(n_splits=3, shuffle=True)
rf_roc_auc_scores = []
rf_average_precision_score = []
rf_f1_score = []

for train_index, test_index in folds.split(np.array(all_df), np.array(y_df)):
    X_train, X_test, y_train, y_test = np.array(all_df)[train_index], np.array(all_df)[test_index], np.array(y_df)[train_index], np.array(y_df)[test_index]
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    model = RandomForestClassifier(n_estimators=300)
    print("model:", model)
    print("X_train legth:", len(X_train))
    print("X_test legth:", len(X_test))
    model.fit(X_train, y_train)
    test_predictions_np = np.array(model.predict(X_test))
    rf_roc_auc_scores.append(roc_auc_score(y_test, test_predictions_np.ravel()))
    rf_average_precision_score.append(average_precision_score(y_test, test_predictions_np))
    rf_f1_score.append(f1_score(y_test, test_predictions_np))
    print("roc_auc_score :", roc_auc_score(y_test, test_predictions_np.ravel()))
    print("rf_average_precision_score: ", average_precision_score(y_test, test_predictions_np))
    print("rf_f1_score: ", f1_score(y_test, test_predictions_np))

print("rf_roc_auc_scores:", rf_roc_auc_scores)
print("Average roc_auc_score:", sum(rf_roc_auc_scores) / len(rf_roc_auc_scores))
print("rf_average_precision_score:", rf_average_precision_score)
print("Average rf_average_precision_score:", sum(rf_average_precision_score) / len(rf_average_precision_score))
print("rf_f1_score:", rf_f1_score)
print("Average rf_f1_score:", sum(rf_f1_score) / len(rf_f1_score))

model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
X_train legth: 69825
X_test legth: 34913
roc_auc_score : 0.8348477228609441
rf_average_precision_score:  0.7889215817026359
rf_f1_score:  0.8282292659675882
model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=